In [1]:
#import libraries
import ee
import os
import geemap
import datetime
import plotly.graph_objects as go
from ipyleaflet import WidgetControl
import ipywidgets as widgets
from ipywidgets import HTML


In [2]:
#define variables

#start session in GEE 
ee.Authenticate()

project = 'ee-vl99956018'
ee.Initialize(project=project)

#before fire date range
pre_start_date = '2025-01-01'
pre_end_date = '2025-01-05'

#after fire date range
pos_start_date = '2025-01-06' 
pos_end_date = '2025-01-16'

#coordinates of interest
latitude = 34.092615
longitude = -118.532875

#Region of interest (ROI) with a buffer of 15km
roi = ee.Geometry.Point(longitude, latitude).buffer(15000)

#palette color for visualization (RGB, NDVI and NBR)
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
}

diff_nbr_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']  # negativo → azul (recuperação), positivo → vermelho (impacto)
}


#Initialize map with center and zoom
Map = geemap.Map(center=[latitude, longitude], zoom=13)


In [3]:
#define functions

def get_all_collections(start_date, end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    # Imagem com bandas selecionadas
    image = collection.select('B8', 'B12', 'B4', 'B3', 'B2').median().clip(roi)

    # NDVI
    ndvi = collection.select(['B8', 'B4']) \
        .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
        .median().clip(roi)

    # NBR
    nbr = collection.select(['B8', 'B12']) \
        .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')) \
        .median().clip(roi)

    return {
        'image': image,
        'ndvi': ndvi,
        'nbr': nbr
    }


def export_to_drive(image, description, folder_name='GEE_exports'):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder_name,
        fileNamePrefix=description.lower().replace(' ', '_'),
        scale=10,  # Sentinel-2 resolution
        region=roi,
        fileFormat='GeoTIFF',
        maxPixels=1e10
    )
    task.start()
    print(f'Started export task for {description}. Check your Google Drive folder "{folder_name}" when complete.')
def create_labels_html():
    """Cria os controles HTML para exibir os rótulos 'ANTES' e 'DEPOIS' no mapa."""
    antes_html = HTML('''<div style="position: absolute; top: 10px; left: 10px; z-index: 1000; 
                        background-color: white; padding: 5px; border-radius: 5px;
                        font-weight: bold; color: black;">ANTES</div>''')
                        
    depois_html = HTML('''<div style="position: absolute; top: 10px; right: 10px; z-index: 1000; 
                         background-color: white; padding: 5px; border-radius: 5px;
                         font-weight: bold; color: black;">DEPOIS</div>''')
    
    return antes_html, depois_html


In [ ]:
#rgb

pre_result = get_all_collections('2025-01-01', '2025-01-05')
pre_image = pre_result['image']

pos_result = get_all_collections('2025-01-06', '2025-01-16')
pos_image = pos_result['image']

left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Adicionar controle de camadas para alternar entre visualizações
Map.addLayer(pre_image, rgb_vis, 'Before Fire', False)
Map.addLayer(pos_image, rgb_vis, 'After Fire', False)

# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")



# Adicionar controles ao mapa
antes_html, depois_html = create_labels_html()
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

Map

Área queimada (ha): {'B12': 4702.045180488614, 'B2': 35934.026037281925, 'B3': 32113.00684965925, 'B4': 18138.10055728667, 'B8': 19471.22234736316}


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [9]:
pre_result = get_all_collections('2025-01-01', '2025-01-05')
pre_image = pre_result['ndvi']

pos_result = get_all_collections('2025-01-06', '2025-01-16')
pos_image = pos_result['ndvi']

left_layer = geemap.ee_tile_layer(pre_image, ndvi_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, ndvi_vis, 'After Fire')

Map.split_map(left_layer=left_layer, right_layer=right_layer)



# Adicionar controle de camadas para alternar entre visualizações
Map.addLayer(pre_image, ndvi_vis, 'Before Fire', False)
Map.addLayer(pos_image, ndvi_vis, 'After Fire', False)

# export_to_drive(pre_image, "before_fire_NDVI_image")
# export_to_drive(pos_image, "after_fire_NDVI_image")

antes_html, depois_html = create_labels_html()
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))



Map

Map(bottom=837395.0, center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [ ]:
#NBR


Map_1 = geemap.Map(center=[latitude, longitude], zoom=13)
Map_2 = geemap.Map(center=[latitude, longitude], zoom=13)


pre_result = get_all_collections('2025-01-01', '2025-01-05')
pre_image = pre_result['nbr']

pos_result = get_all_collections('2025-01-06', '2025-01-16')
pos_image = pos_result['nbr']

dnbr = pre_image.subtract(pos_image)

# 2. Máscara para áreas queimadas (ajuste o limiar se quiser)
burned_mask = dnbr.gt(0.3)

# 3. Calcular área total (em hectares)
pixel_area = burned_mask.multiply(ee.Image.pixelArea()).divide(10000)  # m² para hectares
area_burned = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)

print("Área queimada (ha):", round(area_burned.getInfo()['NBR'],2))


left_layer = geemap.ee_tile_layer(pre_image, nbr_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, nbr_vis, 'After Fire')

Map_1.split_map(left_layer=left_layer, right_layer=right_layer)

# Adicionar controle de camadas para alternar entre visualizações
Map_1.addLayer(pre_image, nbr_vis, 'Before Fire', False)
Map_1.addLayer(pos_image, nbr_vis, 'After Fire', False)

antes_html, depois_html = create_labels_html()
Map_1.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map_1.add_control(WidgetControl(widget=depois_html, position='topright'))

# export_to_drive(pre_image, "before_fire_NBR_image")
# export_to_drive(pos_image, "after_fire_NBR_image")


Map_2.addLayer(dnbr, diff_nbr_vis, 'ΔNBR')

# export_to_drive(dnbr, "Difference NBR")

widgets.VBox([Map_1, Map_2])


Área queimada (ha): 9357.08


In [ ]:
def get_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date, end_date):
    # Carrega coleção Sentinel-2
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterDate(start_date, end_date)
                    .filterBounds(roi)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                    .map(get_ndvi)
                    .map(get_nbr)
                    .select(['NDVI', 'NBR']))

    # Função para extrair NDVI e NBR médios e data como Feature
    def reduce_img(img):
        stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        )
        return ee.Feature(None, {
            'date': img.date().format('YYYY-MM-dd'),
            'NDVI': stats.get('NDVI'),
            'NBR': stats.get('NBR')
        })


    features = collection.map(reduce_img)


    stats_list = features.aggregate_array('date').getInfo()
    ndvi_values = features.aggregate_array('NDVI').getInfo()
    nbr_values = features.aggregate_array('NBR').getInfo()

    # Convert to datetime
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

    # Plot NDVI and NBR curves in the same figure
    fig = go.Figure()
    
    # Add NDVI trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=ndvi_values,
        mode='lines+markers',
        name='NDVI',
        line=dict(color='green', width=2),
        marker=dict(size=8, color='green'),
        yaxis='y1'
    ))
    
    # Add NBR trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=nbr_values,
        mode='lines+markers',
        name='NBR',
        line=dict(color='red', width=2),
        marker=dict(size=8, color='red'),
        yaxis='y1'
    ))

    # Add vertical line at 2025-01-07
    fig.add_vline(
        x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
        line_width=2,
        line_dash="dash",
        line_color="blue",
        annotation_text="First Fire detected",
        annotation_position="top right"
    )

    fig.update_layout(
        title='NDVI and NBR TimeSeries',
        xaxis_title='Date',
        yaxis_title='Index Value',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True, title='Index Value'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_white',
        hovermode='x unified'
    )

    # Save plot
    os.makedirs("results", exist_ok=True)
    fig.write_html("results/ndvi_nbr_plot.html")
    print("Graphic saved at 'results/ndvi_nbr_plot.html'")


if __name__ == "__main__":
    curve_start_date = '2024-11-01'
    curve_end_date = '2025-04-01'

    latitude = 34.092615
    longitude = -118.532875

    # Região de interesse
    roi = ee.Geometry.Point(longitude, latitude).buffer(1000)

    # Run the function
    ndvi_nbr_plot(curve_start_date, curve_end_date)